### Installations, if needed

In [ ]:
# Install the right version of spaCy
!pip install spacy==2.0.12 # Above 2.0.12 doesn't seem work with the neuralcoref resolution (at least 2.0.13 and 2.0.16 don't)

In [ ]:
# Install the large Neural Coref model
!pip install https://github.com/huggingface/neuralcoref-models/releases/download/en_coref_lg-3.0.0/en_coref_lg-3.0.0.tar.gz # This is the coref language model

In [ ]:
# Install dill which we're using for pickling our Named Entity Type (NET) graphs
!pip install dill

### Importing Libraries

In [1]:
import spacy
import en_coref_lg
from spacy import displacy
import pandas as pd
import numpy as np
import networkx as nx
import time
import os
import dill as pickle
from collections import defaultdict

### Skip: Loading and previewing our export from OntoNotes5

In [ ]:
JSON_FILENAME = 'ner_output_1.json'
FILEPATH_TO_JSON = "onto_sql_output/"

onto_import = pd.read_json(FILEPATH_TO_JSON + JSON_FILENAME)

In [ ]:
onto_import.head()

### Skip: Loading the Large Neural Coref spaCy Model

In [2]:
%time
nlp = en_coref_lg.load()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


#### Skip: Using it to loop through dependency parsing for a selection of sentences

In [15]:
# Establish which sentence rows we want to work through right now
SENT_MIN = 0
SENT_MAX = 500

# Check the time and start parsing via spaCy
start_time = time.time()
onto_import["spacy_parse"] = onto_import.loc[SENT_MIN:SENT_MAX,:].apply(lambda x: nlp(x["sentence_string"]), axis=1)

# Calculate the duration 
duration = time.time() - start_time
print("Applying the spaCy pipeline took {0:.2f} seconds".format(duration))

NameError: name 'onto_import' is not defined

### Skip: Viewing some example text, with highlighted named entities

In [ ]:
# Choose an entry integer to see its text and the parse below.
ENTRY = 490

displacy.render(onto_import.loc[ENTRY,"spacy_parse"], jupyter=True, style='ent')

#### Skip: Viewing dependencies

In [ ]:
displacy.render(onto_import.loc[ENTRY,"spacy_parse"], jupyter=True, style='dep')

#### Skip: Identifying some useful attributes of the spaCy tokens

In [ ]:
# This gives us the dependency type

onto_import.loc[490,"spacy_parse"][13].head

In [ ]:
# This gives us the token of its head, from which we can call other attributes of the head.

onto_import.loc[490,"spacy_parse"][13].head.i

In [ ]:
# If we just want the string of the head, that's here:

onto_import.loc[0,"spacy_parse"][32].head.text

In [ ]:
# If we want the index of the head within the sentence (to find when a multi-word NE
# depends on something outside of that NE phrase)

onto_import.loc[0,"spacy_parse"][32].head.i

### Skip: Creating a Dictionary with a Graph for Each NER Type

#### Skip: Establish dictionary to hold graphs and define a function to graph a row 

In [3]:
graphs_dict = dict()

def graph_row(df_row):
    
    NO_OF_GENERATIONS = 2
    
    root_node = str(df_row["ner_type"])

    # Retrieve our Directed Graph for this NE Type or create a new one
    G = graphs_dict.get(root_node, nx.DiGraph())
    
    # For each row, add a node for the Named Entity's type
    G.add_node(root_node)
    node_weight = G.nodes[root_node].get('weight', 0)
    G.node[root_node]['weight'] = node_weight + 1
    G.node[root_node]['xlabel'] = G.node[root_node]['weight']

    # Let's assume the head is the last word of the phrase
    # and get that Token from the spaCy parse:
    
    head_index = df_row["ner_end_word_index"]
    head_of_phrase = df_row["spacy_parse"][head_index]
    
    nodes_needing_head_branches = [(head_of_phrase, root_node)]
    next_head_nodes = []
    nodes_needing_child_branches = [(head_of_phrase, root_node)]
    next_child_nodes = []
    
    current_gen = 1
    
    while current_gen <= NO_OF_GENERATIONS:
        
        for (node, child_label) in nodes_needing_head_branches:
            try: 
                # Get the explanation of its dependency type in this usage
                relation = spacy.explain(node.dep_)

                # If no explanation, revert to the raw dependency type.
                if relation is None:
                    relation = node.dep_

                # Trying to catch and diagnose some problem cases
                elif relation == "punctuation":
                    print("NE '{1}' marked as punctuation in sentence '{0}'".format(df_row["sentence_string"], df_row["ner_string"]))
                    print(" --- ")
                elif relation == "determiner":
                    print("NE '{1}' marked as determiner in sentence '{0}'".format(df_row["sentence_string"], df_row["ner_string"]))
                    print(" --- ")

                # Object of preposition doesn't do much, so let's see what's on the other side of that.
                elif relation == "object of preposition":
                    relation = "head of prep phrase"
                    # move to the preposition so we get its head later on when adding node
                    node = node.head
                    
                print("relation: " + relation)
                
                if relation == 'ROOT':
                    continue
                
                # differentiating head-focused edges from child-focused edges
                intermediary_node_label = "head g{0} {1}".format(current_gen, relation)
                
                print("intermediary node label:" + intermediary_node_label)

                # Add a node for the relation, and connect that to the main entity.
                # Add the weights to the nodes and edges.
                G.add_node(intermediary_node_label)
                relation_weight = G.node[intermediary_node_label].get('weight', 0)
                G.node[intermediary_node_label]['weight'] = relation_weight + 1
                
                # Add a node from the dependency type to the entity's head and add weights
                norm = node.head.norm_
                G.add_node(norm)
                norm_edge_weight = G.node[norm].get('weight', 0)
                G.node[norm]['weight'] = norm_edge_weight + 1
                
                G.add_edge(child_label, intermediary_node_label, label="head")
                
                G.add_edge(intermediary_node_label, norm)
                
                # Add the next round for the next generation
                if node.head != node:
                    next_head_nodes.append((node.head, norm))
                
            except:
                print("passed in head")
                pass
            
            # Move the next round into the queue and clear it
            nodes_needing_head_branches = next_head_nodes
            next_head_nodes = []

            for (node, parent_label) in nodes_needing_child_branches:
                for child in node.children:

                    # Get the relation of the child node to this one.
                    relation = spacy.explain(child.dep_)

                    # If no explanation, revert to the raw dependency type.
                    if relation is None:
                        relation = child.dep_

                    if relation == 'punctuation':
                        continue

                    # Differentiate these relations from head relations
                    # and add the node and its weights
                    intermediary_node_label = "child g{0} {1}".format(current_gen, relation)
                    G.add_node(intermediary_node_label)
                    relation_weight = G.nodes[intermediary_node_label].get('weight', 0)
                    G.node[intermediary_node_label]['weight'] = relation_weight + 1
                    
                    # Add the child as normed, and add its edge and weights
                    child_norm = child.norm_
                    G.add_node(child_norm)
                    leaf_weight = G.node[child_norm].get('weight', 0)
                    G.node[child_norm]['weight'] = leaf_weight + 1

                    # add edge between the parent node and this relation and weights
                    G.add_edge(parent_label, intermediary_node_label, label="child")
                    
                    G.add_edge(intermediary_node_label, child_norm)

                    # Queue up the children for the next generation
                    for childs_child in child.children:
                        next_child_nodes.append((childs_child, child_norm))      
                
                # Move the children into the queue and clear it.
                nodes_needing_child_branches = next_child_nodes
                next_child_nodes = []
        
        # Increment the generation
        current_gen += 1

    graphs_dict[root_node] = G

#### Skip: Applying that to our selected rows of our dataframe

In [ ]:
%time
_ = onto_import.loc[SENT_MIN:SENT_MAX,:].apply(lambda x: graph_row(x), axis=1)

#### Skip: Calculate the Log Probability of the Named Entity Types

In [ ]:
def normalize_as_logp(successor_nodes, G):
    log_total = np.log(sum([G.node[node]['weight'] for node in successor_nodes]))
    for n in successor_nodes:
        G.node[n]['lp'] = np.log(G.node[n]['weight']) - log_total

def calc_lps(G):
    for node in nx.nodes(G):
        if G.successors(node):
            [normalize_as_logp(succ[1], G) for succ in nx.bfs_successors(G, node)]
    return G

In [ ]:
%time

log_graphs_dict = {key:calc_lps(value) for (key,value) in graphs_dict.items()}

#### Skip: Draw the graphs for the Named Entity Types

In [ ]:
%time

for key, value in graphs_dict.items():
    
    graph_filepath = 'NER_Type_Graphs/'
    graph_filename = 'G_{0}'.format(key)

    # Write our graph to DOT format to be read and visualized by GraphViz
    nx.drawing.nx_pydot.write_dot(value, "{0}.dot".format(graph_filepath + graph_filename))
    
    # Write the graphs via neato
    os.system("neato -Tsvg -Goverlap=false -Gsplines=true -Gsep=0.1 {0}.dot -o {0}.svg".format(graph_filepath + graph_filename))

### Graphing a Candidate Named Entity

In [16]:
# Defining helper functions to build the candidate graphs

def reconcile_ents_and_clusters(document_id, doc):
    """"Reconcile the coreference and entities lists into a
        a single dict of graphs to make.
        
        Keys are ent_id strings [document_id]:[start_word_index]:[end_word_index].
        Values are (spaCy.Span, graph_id) tuples."""
    
    # Keys are (start.idx, end.idx) tuples.
    # Values are (spaCy.Span, graph_id) tuples."
    occurence_ind  = {}
    
    if doc._.has_coref:
        cluster_offset = len(doc._.coref_clusters)
        for cluster_idx, cluster in enumerate(doc._.coref_clusters):
            for mention in cluster:
                key = ":".join([document_id,str(mention.start), str(mention.end -1)])
                occurence_ind[key] = (mention, cluster_idx)
                
    # Now let's see if each ent is in there. If not, we'll add it to
    # our cluster list.
    new_cluster_idx = 0
    
    for ent_ind, ent in enumerate(doc.ents):
        key = ":".join([document_id,str(ent.start), str(ent.end -1)])
        try:
            occurence_ind[key]
        except:
            occurence_ind[key] = (ent, cluster_offset + new_cluster_idx)
            new_cluster_idx += 1
    
    print("Length of occurence index for doc {0} is: {1}".format(document_id, len(occurence_ind)))
    
    return occurence_ind

def graph_entity(ent, doc, G, root_node):
    
    NO_OF_GENERATIONS = 2
    
    # Assume the head of the phrase, if it is a phrase, is the last word
    # in the phrase.
    head_of_phrase = ent[-1]
    
    nodes_needing_head_branches = [(head_of_phrase, root_node)]
    next_head_nodes = []
    nodes_needing_child_branches = [(head_of_phrase, root_node)]
    next_child_nodes = []
    
    current_gen = 1
    while current_gen <= NO_OF_GENERATIONS:
        
        for (node, child_label) in nodes_needing_head_branches:
            try: 
                # Get the explanation of its relation arc in this usage
                relation = spacy.explain(node.dep_)
                # If no explanation, revert to the raw dependency type.
                if relation is None:
                    relation = node.dep_
                    
                if relation == 'ROOT':
                    continue

                # Object of preposition doesn't do much, so let's see what's on the other side of that.
                elif relation == "object of preposition":
                    relation = "head of prep phrase"
                    # move to the preposition so we get its head later on when adding node
                    node = node.head
                    
                intermediary_node_label = "head g{0} {1}".format(current_gen, relation)

                # Add a node for the relation, and connect that to the main entity
                G.add_node(intermediary_node_label)
                G.add_edge(child_label, intermediary_node_label, label="head")

                # Add a node from the relation to the entity's head, and connect that
                # to the relation type
                normed_head = node.head.norm_
                #print("adding head node: {0}".format(normed_head))
                G.add_node(normed_head)
                G.add_edge(intermediary_node_label, normed_head)
                
                if node.head != node:
                    next_head_nodes.append((node.head, node.head.text))
            except Exception as ex:
                print("passed in head.\n{0}".format(ex))
                pass
            
        nodes_needing_head_branches = next_head_nodes
        next_head_nodes = []
        
        for (node, parent_label) in nodes_needing_child_branches:
            try:
                for child in node.children:
                    relation = spacy.explain(child.dep_)
                    # If no explanation, revert to the raw dependency type.
                    if relation is None:
                        relation = node.dep_
                        
                    elif relation == 'punctuation':
                        continue
                        
                    intermediary_node_label = "child g{0} {1}".format(current_gen, relation)

                    G.add_node(intermediary_node_label)
                    G.add_edge(parent_label, intermediary_node_label, label="child")
                    
                    normed_child = child.norm_
                    G.add_node(normed_child)
                    #print("adding child node: {0}".format(normed_child))
                    G.add_edge(intermediary_node_label, normed_child)
                    
                    for childs_child in child.children:
                        next_child_nodes.append((childs_child, childs_child.text))
            except Exception as ex:
                print("passed in child.\n{0}".format(ex))
                pass                
        nodes_needing_child_branches = next_child_nodes
        next_child_nodes = []
        
        # Increment the generation
        current_gen += 1
        
    return G
    
def graph_candidates_in_doc(document_id, candidate_text):
    
    doc = nlp(candidate_text)
    
    # Key of clustered_ents is ent_it. Value is (Span, graph_cluter_id)
    clustered_ents = reconcile_ents_and_clusters(document_id, doc)
    
    # Initialize a graph for each clustered_ent
    candidate_graphs = dict()
    
    for ent_id, (ent, graph_idx) in clustered_ents.items():
        
        # Make sure we have our root. No harm done if it already exists.
        # If it's a cluster, we get the Span of the most representative
        # mention in the cluster
        try:
            root_node = doc._.coref_clusters[graph_idx].main.text
        # If it's not, we just use the ent name
        except:
            root_node = ent.text
        
        # Get the cluster's root node and existing graph from previous mentions
        # or create a new tuple of those.
        root_node, G, _ = candidate_graphs.get(graph_idx, (root_node, nx.DiGraph(), graph_idx))
        
        G.add_node(root_node)
        
        # A helper function adds the rest of the graph
        print("\nGraphing entity: {0}".format(ent.text))
        candidate_graphs[graph_idx] = (root_node, graph_entity(ent, doc, G, root_node), graph_idx)
    
    spacy_ent_graphs = {}
    
    # Filter back to just the spaCy ents
    for ent in doc.ents:
        key = ":".join([document_id,str(ent.start), str(ent.end - 1)])
        _, ent_graph_idx = clustered_ents[key]
        spacy_ent_graphs[key] = candidate_graphs[ent_graph_idx]
    
    #graph_entity(ent, doc) for ent in doc.ents]
    return spacy_ent_graphs

#### Skip: Applying the functions on a selected article

In [ ]:
TEST_ENTRY = 90
test_candidate_text = onto_import.loc[TEST_ENTRY,"document_text"]
print(test_candidate_text)

candidate_roots_and_graphs = graph_candidates_in_doc("test_doc_id",test_candidate_text)

In [ ]:
for ent_id in candidate_roots_and_graphs.values():
    print(ent_id)

#### Skip: Draw the candidate graphs for that article

In [ ]:
for key, (root, cand_graph, clust) in candidate_roots_and_graphs.items():
    
    graph_filepath = 'NER_Type_Graphs/'
    graph_filename = 'G_' + str(key)

    # Write our graph to DOT format to be read and visualized by GraphViz
    nx.drawing.nx_pydot.write_dot(cand_graph, "{0}.dot".format(graph_filepath + graph_filename))
    
    os.system("neato -Tsvg -Goverlap=false -Gsplines=true -Gsep=0.1 {0}.dot -o {0}.svg".format(graph_filepath + graph_filename))

## Similarity Between Graphs

In [17]:
def logsumexp(a):
    """Simple re-implementation of scipy.misc.logsumexp."""
    a_max = np.max(a)
    if a_max == -np.inf:
        return -np.inf
    sumexp = np.sum(np.exp(a - a_max))
    return np.log(sumexp) + a_max

def score_relation_and_children(nlp, cand_G, NET_G, cand_parent, net_options, cand_succ_dict, NET_succ_dict, sim_dict):
        
    # If this node has no further children,
    # compare it to its NET options
    if cand_parent not in cand_succ_dict:
        
        sim_scores = list()
                
        cand_token = nlp(cand_parent)[0]
        
        for net_opt in net_options:
            try:
                sim_scores.append(sim_dict[cand_parent][net_opt])
            except:
                score = cand_token.similarity(nlp(net_opt)[0])
                sim_scores.append(score)
                sim_dict[cand_parent][net_opt] = score

        # Get the index of the most similar word
        sim_idx = np.argmax(sim_scores)
                
        # Recover the float value of the winning word's log probability
        sim_lp = float(NET_G.node[net_options[sim_idx]]['lp'])
                
        # Recover the score: log probability of that word under the NET dependency plus
        # the log prob of the similarity score of the most similar word
        similarity_score = sim_lp + np.log(sim_scores[sim_idx])
                        
        return similarity_score
    
    # Otherwise let's score the dependency tags and
    # recursively call this on their children
    else:
        # Prepare to hold scores from multiple branches
        accumulated_scores = []
                
        # Iterate over dependency relations from the parent
        for relation in cand_succ_dict[cand_parent]:
            
            # Proceed if the NET_graph has this relation:
            try:
                # Get the options from the NET graph branching from this relation type
                child_net_options = NET_succ_dict[relation]
            
                # Iterate over the children of each relation
                for cand_child in cand_succ_dict[relation]:
                    score_from_this_node_to_leaves = score_relation_and_children(nlp, cand_G, NET_G, cand_child, child_net_options, cand_succ_dict, NET_succ_dict, sim_dict)
                    if score_from_this_node_to_leaves is not None:
                        accumulated_scores.append(score_from_this_node_to_leaves)
                    
            except Exception as ex:
                print("Exception down candidate {0} branch. No match in NET?\n{1}".format(relation,ex))
        
        # If we have more than an empty list
        if accumulated_scores != list():
            return logsumexp(accumulated_scores)
            
def compare_candidate_to_NET(nlp, candidate_G, candidate_root, NET_G, net_root, sim_dict):
    
    # Calculate the breadth-first search of the candidate graph
    cand_succ_dict = {par:child_list for (par,child_list)in nx.bfs_successors(candidate_G, candidate_root)}

    # Calculate the breadth-first search of the NET graph
    # (For a speed boost we should do this externallay and pass it in.)
    NET_succ_dict = {par:child_list for (par,child_list)in nx.bfs_successors(NET_G, net_root)}
    
    #print("\n\nNET: {0}".format(net_root))
    
    # Run the results of runnign the recursive score_relation_and_children function on the initial roots
    return score_relation_and_children(nlp, candidate_G, NET_G, candidate_root, [], cand_succ_dict, NET_succ_dict, sim_dict)

#### Skip: Calculate similarity between one candidate graph and one NET

In [ ]:
CANDIDATE_GRAPH_NO = 0
NET_TO_COMPARE = 'PRODUCT'

score = compare_candidate_to_NET(nlp, candidate_roots_and_graphs[CANDIDATE_GRAPH_NO][1], candidate_roots_and_graphs[CANDIDATE_GRAPH_NO][0], log_graphs_dict[NET_TO_COMPARE], NET_TO_COMPARE)

print("\nThe comparison score between G_{2} and the {0} graph is {1}".format(NET_TO_COMPARE, score, CANDIDATE_GRAPH_NO))

#### Skip: To manually examine the dicts in the comparison above:

In [ ]:
NET_succ_dict = {par:child_list for (par,child_list)in nx.bfs_successors(log_graphs_dict[NET_TO_COMPARE], NET_TO_COMPARE)}

NET_succ_dict

In [ ]:
cand_succ_dict = cand_succ_dict = {par:child_list for (par,child_list)in nx.bfs_successors(candidate_roots_and_graphs[CANDIDATE_GRAPH_NO][1], candidate_roots_and_graphs[CANDIDATE_GRAPH_NO][0])}

cand_succ_dict

### Loop through documents to generate Y_Pred entitites

In [18]:
### Import the documents with trace strings removed
documents = pd.read_csv('../data/document.csv')

In [19]:
### Import the pickled NET graphs and calculate and add the log probabilities 
%time

with open('../data/LP_NET_Graphs.pkl', 'rb') as file:
  lp_net_graphs = pickle.load(file)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


In [21]:
def predict_max_from_all_nets(cand_root_node, cand_G, NETs_dict, nlp, sim_dict):
    
    # Initialize dict to hold the scores
    similarity_scores_dict = dict()
    
    # Iterate through the NETs
    for net_string, net_graph in NETs_dict.items():
        
        # Score the candidate against each NET
        sim_score = compare_candidate_to_NET(nlp, cand_G, cand_root_node, net_graph, net_string, sim_dict)
        
        # If there is a score for this NET, store it
        if sim_score is not None:
            similarity_scores_dict[net_string] = sim_score
    print(cand_root_node)
    print(similarity_scores_dict)
    
    if len(similarity_scores_dict) > 0:
        return max(similarity_scores_dict, key=(lambda key: similarity_scores_dict[key]))
    else:
        return 'NO_MATCH'

def predict_on_doc(doc_row, NETs_dict, nlp, Y_pred, sim_dict):
    
    cluster_dict = dict()
    
    ent_dict = graph_candidates_in_doc(doc_row["document_id"],doc_row["document"])
    
    for ent_id, (cand_root_node, cand_G, cluster_id) in ent_dict.items():
        try:
            Y_pred.append([ent_id, cluster_dict[cluster_id]])
        except:
            cluster_dict[cluster_id] = predict_max_from_all_nets(cand_root_node, cand_G, NETs_dict, nlp, sim_dict)
            Y_pred.append([ent_id, cluster_dict[cluster_id]])

In [22]:
checkpoint_1 = time.time()

DOC_MIN = 0 
DOC_MAX = 1

# Build the graphs of candidates from the documents.
Y_pred = list()
sim_dict = defaultdict(lambda: dict())

# Loading a spaCy model but disabling parsers to speed up similarity measurements
sim_nlp = spacy.load('en_core_web_lg', disable=['parser', 'tagger', 'ner', 'neuralcoref'])

documents.loc[DOC_MIN:DOC_MAX,:].apply(lambda x: predict_on_doc(x, lp_net_graphs, sim_nlp, Y_pred, sim_dict), axis=1)

print("That took: {0} seconds".format(round(time.time() - checkpoint_1, 4)))


Length of occurence index for doc bc/cctv/00/cctv_0000@0000@cctv@bc@en@on is: 490

Graphing entity: Hong Kong

Graphing entity: Hong Kong

Graphing entity: Hong Kong

Graphing entity: Hong Kong

Graphing entity: Hong Kong

Graphing entity: Hong Kong

Graphing entity: Hong Kong

Graphing entity: Disney

Graphing entity: Disney

Graphing entity: it

Graphing entity: Hong Kong Disneyland

Graphing entity: Hong Kong Disneyland

Graphing entity: Lantau Island

Graphing entity: Lantau Island

Graphing entity: this area

Graphing entity: its

Graphing entity: this area

Graphing entity: Hong Kong International Airport

Graphing entity: its

Graphing entity: Disney

Graphing entity: Disney

Graphing entity: Disney

Graphing entity: Disney

Graphing entity: its

Graphing entity: Disney

Graphing entity: Disney

Graphing entity: the station for Disney

Graphing entity: it

Graphing entity: the Disney subway station

Graphing entity: the park

Graphing entity: the park

Graphing entity: this myst

the summer of 2005
{'ORDINAL': -8.340374310782643, 'PERSON': -10.481477441586515, 'DATE': -9.182163472029838, 'GPE': -8.220133957151859, 'NORP': -9.630467998063857, 'CARDINAL': -8.41141576345885, 'ORG': -9.42696602038473, 'EVENT': -5.433003918801125, 'WORK_OF_ART': -6.815759243370108, 'MONEY': -8.563451731009984, 'LANGUAGE': -5.499546467349677, 'FAC': -6.731863030478861, 'LOC': -7.756195343948118, 'QUANTITY': -7.364984680078957, 'LAW': -6.452867433325661, 'TIME': -6.280804689235969, 'PRODUCT': -7.182179890353676, 'PERCENT': -7.18724283397543}
Exception down candidate child g1 nsubj branch. No match in NET?
'child g1 nsubj'
Exception down candidate child g1 dobj branch. No match in NET?
'child g1 dobj'
Exception down candidate child g1 pobj branch. No match in NET?
'child g1 pobj'
Exception down candidate child g1 nsubj branch. No match in NET?
'child g1 nsubj'
Exception down candidate child g1 dobj branch. No match in NET?
'child g1 dobj'
Exception down candidate head g2 attribute bran

several years
{'ORDINAL': -0.7124441612987864, 'PERSON': -5.828618488126013, 'DATE': -6.579292976201756, 'GPE': -3.1541536518280346, 'NORP': -4.204709625779302, 'CARDINAL': -4.090229989700022, 'ORG': -1.9942340136568921, 'EVENT': -1.5952206737753227, 'WORK_OF_ART': -3.7185894261758436, 'MONEY': -4.498537221473478, 'LANGUAGE': -2.428241921321462, 'FAC': -1.6518802088201954, 'LOC': -4.504645514639051, 'QUANTITY': -3.7000483443482297, 'LAW': -4.1276308254108764, 'TIME': -2.974440409636045, 'PRODUCT': -1.0835491227645404, 'PERCENT': -3.590774464693373}
Exception down candidate child g1 pobj branch. No match in NET?
'child g1 pobj'
Exception down candidate child g1 pobj branch. No match in NET?
'child g1 pobj'
Exception down candidate child g1 pobj branch. No match in NET?
'child g1 pobj'
Exception down candidate child g1 pobj branch. No match in NET?
'child g1 pobj'
Exception down candidate child g1 pobj branch. No match in NET?
'child g1 pobj'
Exception down candidate child g1 pobj branch

Exception down candidate child g1 pobj branch. No match in NET?
'child g1 pobj'
Exception down candidate child g1 pobj branch. No match in NET?
'child g1 pobj'
Exception down candidate child g1 pobj branch. No match in NET?
'child g1 pobj'
Exception down candidate child g1 pobj branch. No match in NET?
'child g1 pobj'
Exception down candidate child g1 pobj branch. No match in NET?
'child g1 pobj'
Exception down candidate child g1 pobj branch. No match in NET?
'child g1 pobj'
Disney Corporation
{'ORDINAL': -6.840377890870639, 'PERSON': -6.500309257190155, 'DATE': -7.492053207529933, 'GPE': -5.903866487789997, 'NORP': -6.38370454507394, 'CARDINAL': -4.16460736495557, 'ORG': -6.222717981385571, 'EVENT': -3.8423033221011176, 'WORK_OF_ART': -3.52714147010691, 'MONEY': -4.146868746980566, 'LANGUAGE': -5.834069696524667, 'FAC': 0.0, 'LOC': -6.146757431514017, 'QUANTITY': -3.3715949819835966, 'LAW': -6.403522483988405, 'TIME': -5.694405312843604, 'PRODUCT': -6.17458706517665, 'PERCENT': -4.535

Chinese
{'ORDINAL': -7.787382026484701, 'PERSON': -10.612639250554105, 'DATE': -7.2461133696229565, 'GPE': -9.360674584129596, 'NORP': -8.7157161275482, 'CARDINAL': -8.899409597957089, 'ORG': -11.071264449709311, 'EVENT': -7.031800458542474, 'WORK_OF_ART': -7.658360063914184, 'MONEY': -7.40806446847255, 'LANGUAGE': -5.527787028834967, 'FAC': -7.123927080318834, 'LOC': -7.825333634350697, 'QUANTITY': -7.372870076559518, 'LAW': -6.756379052893532, 'TIME': -8.102501238621516, 'PRODUCT': -7.368775449473854, 'PERCENT': -7.129002837542172}
first
{'ORDINAL': -7.787382026484701, 'PERSON': -10.612639250554105, 'DATE': -7.2461133696229565, 'GPE': -9.360674584129596, 'NORP': -8.7157161275482, 'CARDINAL': -8.899409597957089, 'ORG': -11.071264449709311, 'EVENT': -7.031800458542474, 'WORK_OF_ART': -7.658360063914184, 'MONEY': -7.40806446847255, 'LANGUAGE': -5.527787028834967, 'FAC': -7.123927080318834, 'LOC': -7.825333634350697, 'QUANTITY': -7.372870076559518, 'LAW': -6.756379052893532, 'TIME': -8.1

...

Exception down candidate head g1 case branch. No match in NET?
'head g1 case'
Exception down candidate head g1 case branch. No match in NET?
'head g1 case'
Exception down candidate head g1 case branch. No match in NET?
'head g1 case'
Exception down candidate head g1 case branch. No match in NET?
'head g1 case'
Exception down candidate head g1 case branch. No match in NET?
'head g1 case'
Nie Rongzhen 's
{'ORDINAL': -inf, 'PERSON': -7.766840537085513, 'DATE': -inf, 'GPE': -inf, 'NORP': -inf, 'CARDINAL': -inf, 'ORG': -inf, 'WORK_OF_ART': -inf, 'FAC': -inf, 'LOC': -inf, 'LAW': -inf, 'TIME': -inf, 'PRODUCT': -inf}
Exception down candidate child g1 nsubj branch. No match in NET?
'child g1 nsubj'
Exception down candidate child g1 nsubj branch. No match in NET?
'child g1 nsubj'
Exception down candidate child g1 nsubj branch. No match in NET?
'child g1 nsubj'
Exception down candidate child g1 nsubj branch. No match in NET?
'child g1 nsubj'
Exception down candidate child g1 nsubj branch. No matc

two hundred thousand
{'ORDINAL': -4.238112864041732, 'PERSON': -4.699963222848246, 'DATE': -5.217972487138197, 'GPE': -2.300133910046505, 'NORP': -4.594025602594197, 'CARDINAL': -7.640379492902802, 'ORG': -5.556387113287146, 'EVENT': -2.653036879817815, 'WORK_OF_ART': -2.612248514191666, 'MONEY': -3.0691435747436167, 'LANGUAGE': -1.4768968764509287, 'FAC': -2.080251553269717, 'LOC': -1.120964361123725, 'QUANTITY': -0.8736447398979514, 'LAW': -3.1978777278287023, 'TIME': -2.40693559490768, 'PRODUCT': -3.7975695698332355, 'PERCENT': -3.7180969915354654}
Exception down candidate child g1 nsubj branch. No match in NET?
'child g1 nsubj'
Exception down candidate child g1 nsubj branch. No match in NET?
'child g1 nsubj'
Exception down candidate child g1 nsubj branch. No match in NET?
'child g1 nsubj'
Exception down candidate child g1 nsubj branch. No match in NET?
'child g1 nsubj'
Exception down candidate child g1 nsubj branch. No match in NET?
'child g1 nsubj'
Exception down candidate child g

Exception down candidate child g1 adverbial modifier branch. No match in NET?
'child g1 adverbial modifier'
Britain
{'ORDINAL': -2.758685480330142, 'PERSON': -4.927631576733773, 'DATE': -7.401314436353913, 'GPE': -3.629609149615221, 'NORP': -4.483417072676401, 'CARDINAL': -6.679279142701343, 'ORG': -3.14876271299977, 'EVENT': -2.2735235633715956, 'WORK_OF_ART': -2.842228289945659, 'MONEY': -3.6544491334430993, 'LANGUAGE': -1.4920607253930094, 'FAC': -4.053066725514448, 'LOC': -2.213003028028458, 'QUANTITY': -5.018648440958388, 'LAW': -2.772588722239781, 'TIME': -5.308511037632453, 'PRODUCT': -2.6695277661038883, 'PERCENT': -4.569453146556097}
Exception down candidate child g1 noun phrase as adverbial modifier branch. No match in NET?
'child g1 noun phrase as adverbial modifier'
Exception down candidate head g1 adverbial modifier branch. No match in NET?
'head g1 adverbial modifier'
Exception down candidate head g1 adverbial modifier branch. No match in NET?
'head g1 adverbial modifier'

Exception down candidate child g1 poss branch. No match in NET?
'child g1 poss'
Exception down candidate child g1 poss branch. No match in NET?
'child g1 poss'
Exception down candidate child g1 poss branch. No match in NET?
'child g1 poss'
Exception down candidate child g1 poss branch. No match in NET?
'child g1 poss'
Exception down candidate child g1 poss branch. No match in NET?
'child g1 poss'
Exception down candidate child g1 poss branch. No match in NET?
'child g1 poss'
Exception down candidate child g1 poss branch. No match in NET?
'child g1 poss'
Exception down candidate head g1 possession modifier branch. No match in NET?
'head g1 possession modifier'
Exception down candidate child g1 poss branch. No match in NET?
'child g1 poss'
Exception down candidate head g1 possession modifier branch. No match in NET?
'head g1 possession modifier'
Exception down candidate child g1 poss branch. No match in NET?
'child g1 poss'
Exception down candidate child g1 poss branch. No match in NET?


Exception down candidate child g1 dobj branch. No match in NET?
'child g1 dobj'
Exception down candidate child g1 dobj branch. No match in NET?
'child g1 dobj'
Exception down candidate child g1 dobj branch. No match in NET?
'child g1 dobj'
Exception down candidate child g1 dobj branch. No match in NET?
'child g1 dobj'
Exception down candidate child g1 dobj branch. No match in NET?
'child g1 dobj'
Exception down candidate child g1 dobj branch. No match in NET?
'child g1 dobj'
the %
{'ORDINAL': -0.1728489424013245, 'PERSON': -2.548750098038204, 'DATE': -4.783552166450651, 'GPE': 0.00735444443701827, 'NORP': -2.450825560794126, 'CARDINAL': -0.8327142823622408, 'ORG': -0.19610882931467621, 'EVENT': -0.07206244036579133, 'WORK_OF_ART': -0.5443128970104564, 'MONEY': -0.32296739169389116, 'LANGUAGE': -0.16152196470253072, 'FAC': -0.008720619417046318, 'LOC': -0.025563623878384657, 'QUANTITY': -0.9222872395664672, 'LAW': 0.018972813293703535, 'TIME': -0.4150050970439683, 'PRODUCT': -0.06538347

#### Confirming the number of entities match for the first two documents

In [24]:
len(Y_pred)

714

In [25]:
doc0 = nlp(documents.loc[0,"document"])
doc1 = nlp(documents.loc[1,"document"])

len(doc0.ents) + len(doc1.ents)

714

### Messing with the Y_Pred and Y_True dataframes

In [26]:
ypred_df = pd.DataFrame(Y_pred, columns = ['entity_id', 'prediction']).set_index('entity_id')

In [29]:
ypred_df.to_csv('../data/Y_pred.csv')

In [30]:
with open('../data/Y_true.pkl', 'rb') as file:
  ytrue_df = pickle.load(file)

In [42]:
ymerged_df = ypred_df.merge(ytrue_df,how='inner',on='entity_id')
ymerged_df.shape

(61, 7)

In [43]:
ymerged_df

,prediction,document_id,type,sentence_index,start_word_index,end_word_index,string
entity_id,,,,,,,
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:1:4,EVENT,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,DATE,0,1,4,the summer of 2005
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:23:24,WORK_OF_ART,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,GPE,0,23,24,Hong Kong
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:41:42,WORK_OF_ART,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,GPE,1,14,15,Hong Kong
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:54:54,PERCENT,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,ORDINAL,2,3,3,fifth
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:55:55,ORDINAL,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,ORG,2,4,4,Disney
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:70:70,ORDINAL,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,ORG,3,5,5,Disney
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:82:83,ORDINAL,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,DATE,4,3,4,several years
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:108:109,WORK_OF_ART,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,GPE,4,29,30,Hong Kong
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:133:135,QUANTITY,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,DATE,6,7,9,two years ago


In [45]:
ytrue_df.filter(like="bc/cctv/00/cctv_0000@0000@cctv@bc@en@on",axis=0).head()

,document_id,type,sentence_index,start_word_index,end_word_index,string
entity_id,,,,,,
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:1:4,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,DATE,0,1,4,the summer of 2005
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:23:24,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,GPE,0,23,24,Hong Kong
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:41:42,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,GPE,1,14,15,Hong Kong
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:54:54,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,ORDINAL,2,3,3,fifth
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:55:55,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,ORG,2,4,4,Disney


In [46]:
matches = ymerged_df.loc[:,["prediction","type"]].apply(lambda x: x["prediction"] == x["type"], axis=1)
accuracy = sum(matches) / ymerged_df.shape[0]
print("Accuracy: {0}".format(accuracy))

Accuracy: 0.06557377049180328


In [55]:
ytrue_df.groupby("type").size()

type
CARDINAL       13638
DATE           23807
EVENT           1273
FAC             1440
GPE            28142
LANGUAGE         415
LAW              568
LOC             2692
MONEY           6429
NORP           11610
ORDINAL         2738
ORG            29963
PERCENT         4866
PERSON         27351
PRODUCT         1297
QUANTITY        1583
TIME            2290
WORK_OF_ART     1652
dtype: int64